In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("reg_cleaned_data.csv")
df.columns

Index(['bad_flag', 'loan_amt_requested', 'mort_due', 'current_property_value',
       'loan_reason', 'job', 'years_on_job', 'no_of_derog', 'no_of_delinq',
       'age_of_oldest_cl', 'no_of_recent_credit_inquiries', 'no_of_cl',
       'debt_to_income_ratio', 'loan_to_value_ratio', 'cl_delinquency_ratio',
       'approx_income'],
      dtype='object')

In [3]:
# Create customer_id in the format CUST0001, CUST0002, ..., CUST5796
df['customer_id'] = ['CUST' + str(i).zfill(4) for i in range(1, len(df) + 1)]

# Move customer_id to the front of the DataFrame
df = df[['customer_id'] + [col for col in df.columns if col != 'customer_id']]

In [4]:
df.tail()

,customer_id,bad_flag,loan_amt_requested,mort_due,current_property_value,loan_reason,job,years_on_job,no_of_derog,no_of_delinq,age_of_oldest_cl,no_of_recent_credit_inquiries,no_of_cl,debt_to_income_ratio,loan_to_value_ratio,cl_delinquency_ratio,approx_income
5791,CUST5792,0,88900,57264.0,90185.0,DebtCon,Other,16.0,0,0,221.808718,0,16,36.112347,0.985752,0.0,158571.8058
5792,CUST5793,0,89000,54576.0,92937.0,DebtCon,Other,16.0,0,0,208.692070,0,15,35.859971,0.957638,0.0,152191.9817
5793,CUST5794,0,89200,54045.0,92924.0,DebtCon,Other,15.0,0,0,212.279697,0,15,35.556590,0.959924,0.0,151997.1382
5794,CUST5795,0,89800,50370.0,91861.0,DebtCon,Other,14.0,0,0,213.892709,0,16,34.340882,0.977564,0.0,146676.4874
5795,CUST5796,0,89900,48811.0,88934.0,DebtCon,Other,15.0,0,0,219.601002,0,16,34.571519,1.010862,0.0,141188.4732


In [5]:
# Add new combined Loan to Value Ratio(was loan_amt_requested/current property value) to (loan_amt_requested+mort_due)/current property value
df['combined_ltv_ratio'] = (df['mort_due'] + df['loan_amt_requested']) / df['current_property_value']

# remove 15 inf values in combined_ltv_ratio
import numpy as np

# Replace infinite values with 0 in the full dataset
df.replace([np.inf, -np.inf], 0, inplace=True)

In [6]:
# Define more interpretable income thresholds
bins = [0, 50000, 150000, df['approx_income'].max()]
labels = ['LOW', 'MED', 'HIGH']

# Apply custom binning
df['income_groups'] = pd.cut(df['approx_income'], bins=bins, labels=labels, include_lowest=True)

# Optional: Check the bin stats
print(df.groupby('income_groups')['approx_income'].agg(['count', 'mean', 'std']))

# Step 2: Group by income group and bad_flag, then count
bad_flag_counts = df.groupby(['income_groups', 'bad_flag'])['customer_id'].count().unstack(fill_value=0)

# Step 3: Rename columns for clarity
bad_flag_counts.columns = ['Good_Loans (0)', 'Bad_Loans (1)']

# Step 4: Display the result
print(bad_flag_counts)

               count           mean            std
income_groups                                     
LOW              240   31478.210189   13859.172582
MED             1753  107538.582616   28133.450145
HIGH            3803  292500.238509  315155.377025
               Good_Loans (0)  Bad_Loans (1)
income_groups                               
LOW                       143             97
MED                      1320            433
HIGH                     3193            610


C:\Users\akara\AppData\Local\Temp\ipykernel_9628\3534323895.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(df.groupby('income_groups')['approx_income'].agg(['count', 'mean', 'std']))
C:\Users\akara\AppData\Local\Temp\ipykernel_9628\3534323895.py:12: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  bad_flag_counts = df.groupby(['income_groups', 'bad_flag'])['customer_id'].count().unstack(fill_value=0)


In [7]:
# Get pre one-hot encoded dataset
df.to_csv('cleaned_non_encoded_dataset.csv', index=False)

In [8]:
# Step 1: Define categorical columns for one-hot encoding
categorical_cols = ['loan_reason', 'job', 'income_groups']

# Step 2: One-hot encode with lowercase-friendly prefixes
df_encoded = pd.get_dummies(df, columns=categorical_cols, prefix=categorical_cols, drop_first=False)

# Step 3: Convert boolean dummies to integer format
df_encoded = df_encoded.astype({col: int for col in df_encoded.columns if df_encoded[col].dtype == 'bool'})

# Step 4: Rename encoded columns using naming convention
rename_map = {
    'loan_reason_DebtCon': 'loan_reason_debtCon',
    'loan_reason_HomeImp': 'loan_reason_homeImp',
    'loan_reason_NonProvided': 'loan_reason_nonProvided',
    'job_NonProvided': 'job_nonProvided',
    'job_Office': 'job_office',
    'job_Other': 'job_other',
    'job_ProfExe': 'job_profExe',
    'job_Sales': 'job_sales',
    'job_Self': 'job_self',
    'job_Mgr': 'job_mgr',
    'job_CustServ': 'job_custServ',
    'income_groups_LOW': 'income_group_low',
    'income_groups_MED': 'income_group_med',
    'income_groups_HIGH': 'income_group_high',
    'loan_to_value_ratio': 'ltv_ratio'
}
df_encoded.rename(columns={k: v for k, v in rename_map.items() if k in df_encoded.columns}, inplace=True)

# Step 5: Combine selected dummy columns
df_encoded['income_group_lowMed'] = df_encoded.get('income_group_low', 0) + df_encoded.get('income_group_med', 0)
df_encoded['job_other_nonProvided'] = df_encoded.get('job_other', 0) + df_encoded.get('job_nonProvided', 0)
df_encoded['loan_reason_debtCon_np'] = df_encoded.get('loan_reason_nonProvided', 0) + df_encoded.get('loan_reason_debtCon', 0)

# Step 6: Drop merged original dummy columns
df_encoded.drop(columns=[
    col for col in [
        'income_group_low', 'income_group_med', 'ltv_ratio',
        'job_other', 'job_nonProvided',
        'loan_reason_nonProvided', 'loan_reason_debtCon'
    ] if col in df_encoded.columns
], inplace=True)

# Step 7: Preview resulting group columns
print(df_encoded[['income_group_lowMed', 'job_other_nonProvided', 'loan_reason_debtCon_np']].head())


   income_group_lowMed  job_other_nonProvided  loan_reason_debtCon_np
0                    1                      1                       0
1                    0                      1                       0
2                    1                      1                       0
3                    0                      0                       0
4                    1                      1                       0


In [9]:
# Temporarily show all columns
pd.set_option('display.max_columns', None)
df_encoded.head(10)

,customer_id,bad_flag,loan_amt_requested,mort_due,current_property_value,years_on_job,no_of_derog,no_of_delinq,age_of_oldest_cl,no_of_recent_credit_inquiries,no_of_cl,debt_to_income_ratio,cl_delinquency_ratio,approx_income,combined_ltv_ratio,loan_reason_homeImp,job_mgr,job_office,job_profExe,job_sales,job_self,income_group_high,income_group_lowMed,job_other_nonProvided,loan_reason_debtCon_np
0,CUST0001,1,1100,25860.00000,39025.0,10.5,0,0,94.366667,1,9,35.245224,0.000000,73371.64261,0.690839,1,0,0,0,0,0,0,1,1,0
1,CUST0002,1,1300,70053.00000,68400.0,7.0,0,2,121.833333,0,14,35.245224,0.142857,198758.84300,1.043173,1,0,0,0,0,0,1,0,1,0
2,CUST0003,1,1500,13500.00000,16700.0,4.0,0,0,149.466667,1,10,35.245224,0.000000,38303.06169,0.898204,1,0,0,0,0,0,0,1,1,0
3,CUST0004,0,1700,97800.00000,112000.0,3.0,0,0,93.333333,0,14,36.170905,0.000000,270383.05700,0.888393,1,0,1,0,0,0,1,0,0,0
4,CUST0005,1,1700,30548.00000,40320.0,9.0,0,0,101.466002,1,8,37.113614,0.000000,82309.41984,0.799802,1,0,0,0,0,0,0,1,1,0
5,CUST0006,1,1800,48649.00000,57037.0,5.0,3,2,77.100000,1,17,35.245224,0.117647,138030.04800,0.884496,1,0,0,0,0,0,0,1,1,0
6,CUST0007,1,1800,28502.00000,43034.0,11.0,0,0,88.766030,0,8,36.884894,0.000000,77272.82591,0.704141,1,0,0,0,0,0,0,1,1,0
7,CUST0008,1,2000,32700.00000,46740.0,3.0,0,2,216.933333,1,12,35.245224,0.166667,92778.52720,0.742405,1,0,0,0,0,0,0,1,1,0
8,CUST0009,1,2000,43280.13929,62250.0,16.0,0,0,115.800000,0,13,35.764058,0.000000,121015.73830,0.727392,1,0,0,0,1,0,0,1,0,0
9,CUST0010,1,2000,20627.00000,29800.0,11.0,0,1,122.533333,1,9,36.170905,0.111111,57026.49608,0.759295,1,0,1,0,0,0,0,1,0,0


In [10]:
# Save to CSV
df_encoded.to_csv('cleaned_and_encoded_dataset.csv', index=False)

In [33]:
# # Step 1: Bin approx_income into quantile-based categories
# df['income_bin'] = pd.qcut(df['approx_income'], q=3, labels=['LOW', 'MED', 'HIGH'])

# # Step 2: Calculate mean and standard deviation for each bin
# income_bin_stats = df.groupby('income_bin')['approx_income'].agg(['mean', 'std']).reset_index()

# # Step 3: Display the result
# print(income_bin_stats)
